In [ ]:
!pip install -q underthesea
# !pip install -q transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.0 MB/s eta 0:00:00


In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,AdamW, AutoModelForCausalLM
from torch.optim import Adam
import gdown
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm
from huggingface_hub import login
from google.colab import files
import shutil
import gdown
from typing import List
#from underthesea import sent_tokenize
from nltk import word_tokenize as lib_tokenizer
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#processing dataset

Dữ liệu

In [2]:
gdown.download('https://drive.google.com/uc?id=1Ebtb6ItULnfFqz0qYKdVkaror-GvuNJr')# training
gdown.download('https://drive.google.com/uc?id=1SA4i75wppxd2Qf6E3-8Qe2quIPlYMS9d')# dev

Downloading...
From: https://drive.google.com/uc?id=1Ebtb6ItULnfFqz0qYKdVkaror-GvuNJr
To: /content/vi_squad.csv
100%|██████████| 93.3M/93.3M [00:00<00:00, 102MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SA4i75wppxd2Qf6E3-8Qe2quIPlYMS9d
To: /content/vi_squad_dev.csv
100%|██████████| 11.7M/11.7M [00:00<00:00, 154MB/s]


'vi_squad_dev.csv'

In [3]:
df = pd.read_csv('/content/vi_squad.csv')
# dev = pd.read_csv('/content/vi_squad_dev.csv')

In [4]:
df.head()

,context,question,answer
0,"Architecturally, trường học có một nhân vật Cô...",Ai đã làm Trinh nữ Mary bị cáo buộc xuất hiện ...,Saint Bernadette Soubirous
1,"Architecturally, trường học có một nhân vật Cô...",Cái gì ở trước tòa nhà chính của Notre Dame?,một bức tượng đồng của Chúa Kitô
2,"Architecturally, trường học có một nhân vật Cô...",Thánh đường của trái tim thiêng liêng tại Notr...,Tòa nhà chính
3,"Architecturally, trường học có một nhân vật Cô...",Glimpses ở Notre Dame là gì?,một nơi Marian của cầu nguyện và phản ánh
4,"Architecturally, trường học có một nhân vật Cô...",Cái gì ngồi trên đỉnh của tòa nhà chính tại No...,một bức tượng vàng của Trinh Mary


#Dataset/Dataloader

In [4]:
SEP_TOKEN = '<sep>'

In [6]:
class TextDataset(Dataset):
    def __init__(self, data, tokenizer, src_max_token_len=128, tgt_max_token_len=128):
        self.data = data
        self.tokenizer = tokenizer
        self.src_max_token_len = src_max_token_len
        self.tgt_max_token_len = tgt_max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_row = self.data.iloc[idx]
        input_encoding = self.tokenizer(data_row['context'],
                                        truncation=True,
                                        max_length=self.src_max_token_len,
                                        padding='max_length',
                                        return_tensors='pt')

        output_encoding = self.tokenizer('{} {} {}'.format(data_row['question'], SEP_TOKEN,data_row['answer']),
                                          truncation=True,
                                          max_length=self.tgt_max_token_len,
                                          padding='max_length',
                                          return_tensors='pt')

        labels = output_encoding['input_ids']
        labels[labels == 0] = -100

        return dict(
            context=data_row['context'],
            question=data_row['question'],
            answer=data_row['answer'],
            input_ids=input_encoding['input_ids'].flatten(),
            attention_mask=input_encoding['attention_mask'].flatten(),
            labels=labels.flatten()
        )


In [7]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

In [8]:
train_pairs, test_pairs = train_test_split(df, test_size=0.2, random_state=42)
train_pairs, dev_pairs = train_test_split(train_pairs, test_size=0.2, random_state=42)


In [9]:
train_dataset = TextDataset(train_pairs, tokenizer)
dev_dataset = TextDataset(dev_pairs, tokenizer)
test_dataset = TextDataset(test_pairs, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Train

In [10]:
epochs = 1
lr = 5e-5

In [42]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)


shared.weight True
encoder.block.0.layer.0.SelfAttention.q.weight True
encoder.block.0.layer.0.SelfAttention.k.weight True
encoder.block.0.layer.0.SelfAttention.v.weight True
encoder.block.0.layer.0.SelfAttention.o.weight True
encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight True
encoder.block.0.layer.0.layer_norm.weight True
encoder.block.0.layer.1.DenseReluDense.wi.weight True
encoder.block.0.layer.1.DenseReluDense.wo.weight True
encoder.block.0.layer.1.layer_norm.weight True
encoder.block.1.layer.0.SelfAttention.q.weight True
encoder.block.1.layer.0.SelfAttention.k.weight True
encoder.block.1.layer.0.SelfAttention.v.weight True
encoder.block.1.layer.0.SelfAttention.o.weight True
encoder.block.1.layer.0.layer_norm.weight True
encoder.block.1.layer.1.DenseReluDense.wi.weight True
encoder.block.1.layer.1.DenseReluDense.wo.weight True
encoder.block.1.layer.1.layer_norm.weight True
encoder.block.2.layer.0.SelfAttention.q.weight True
encoder.block.2.layer.0.SelfAttenti

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



for name, param in model.named_parameters():
    if 'final_layer' in name or 'decoder' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)



best_dev_loss = float('inf')
for epoch in range(epochs):
    model.train()
    train_loop = tqdm(train_dataloader, leave=True)
    for batch in train_loop:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        train_loop.set_description(f'Epoch {epoch}')
        train_loop.set_postfix(loss=loss.item())

    model.eval()
    dev_loss = 0
    with torch.no_grad():
        for batch in dev_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            dev_loss += outputs.loss.item()

    dev_loss /= len(dev_dataloader)

    if dev_loss < best_dev_loss:
        best_dev_loss = dev_loss
        model.save_pretrained("my_vit5_finetuned")
        tokenizer.save_pretrained("my_vit5_finetuned")

model = AutoModelForSeq2SeqLM.from_pretrained("my_vit5_finetuned")
tokenizer = AutoTokenizer.from_pretrained("my_vit5_finetuned")
model.to(device)
model.eval()

test_loss = 0
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        test_loss += outputs.loss.item()

test_loss /= len(test_dataloader)
print(f'Test loss: {test_loss}')


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 0: 100%|██████████| 1697/1697 [35:41<00:00,  1.26s/it, loss=1.48]


Test loss: 1.8048654309773848


Download before crash


In [12]:
model.save_pretrained("my_vit5_finetuned")
tokenizer.save_pretrained("my_vit5_finetuned")

shutil.make_archive('/content/my_vit5_finetuned', 'zip', 'my_vit5_finetuned')# zip

files.download('/content/my_vit5_finetuned.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
input_texts = """
1. Ngôn ngữ đánh dấu siêu văn bản
Thông thường, một website gồm một số trang web tĩnh và một số trang web động. Trang web tĩnh có nội dung không thay đổi mỗi khi người dùng truy cập. Ngược lại, nội dung trang web động có thể thay đổi tuỳ theo yêu cầu của người dùng. Tìm hiểu xong chủ đề này, em sẽ tạo được các trang web tĩnh.
Có nhiều cách để tạo trang web. Bên cạnh cách sử dụng phần mềm có sẵn như: Dreamweaver, Mobirise,.., em có thể tạo trang web bằng ngôn ngữ chuyên dụng. Ngôn ngữ đánh dấu siêu văn bản (HyperText Markup Language - HTML) là ngôn ngữ chuyên dụng dùng để tạo trang web.
Thông qua các phần tử của mình, HTML cho phép khai báo các thành phần của trang web như tiêu đề mục, đoạn văn, bảng biểu, hình ảnh, âm thanh và các siêu liên kết,...

Phần đầu của văn bản được xác định thông qua phần tử head. Nội dung phần tử head được viết trong cặp thẻ mở <head> và thẻ đóng </head>, dùng để khai báo tiêu đề trang web, các siêu dữ liệu mô tả thông tin về trang web. Siêu dữ liệu có thể gồm bảng mã kí tự, từ khóa tìm kiếm và các liên kết đến tài nguyên khác nhằm chỉ dẫn trình duyệt web trong việc phân tích và hiển thị kết quả. Tiêu đề trang web được viết trong cặp thẻ mở <title> và thẻ đóng </title> và sẽ được hiển thị trên tiêu đề của cửa sổ trình duyệt web. Các thông tin khác không hiến thị trong màn hình cửa sổ trình duyệt web.
Phần thân của văn bản được xác định thông qua phần tử body. Nội dung của phần tử body được viết trong cặp thẻ mở <body> và thẻ đóng </body> sẽ được hiển thị trong màn hình của cửa số trình duyệt web.
Thông thường, dòng đầu tiên của văn bản HTML là một chỉ dẫn cung cấp thông tin phiên bản HTML được sử dụng.

3. Thực hành tạo trang web đơn giản
Sử dụng phần mềm Sublime Text soạn văn bản HTML thuận tiện hơn so với việc dùng các phần mềm soạn văn bản được cài sẵn trên máy tính. Phần mềm Sublime Text cung cấp một số tính năng như: sử dụng màu sắc để phân biệt các phần tử, tự động điền thẻ đóng cho phần tử được khai báo, đánh số dòng văn bản HTML,…

Yêu cầu 1: Cài đặt phần mềm Sublime Text.
Hướng dẫn thực hiện:
Bước 1. Truy cập trang web https://sublimetext.com, chọn mục Download.
Bước 2. Chọn phiên bản phù hợp với hệ điều hành đang sử dụng và tải về máy tính.
Bước 3. Nháy đúp chuột vào tên tệp đã được tải về ở Bước 2. Khi trên màn hình xuất hiện cửa sổ với thông báo "Completing the Sublime Text Setup Wizard", việc cài đặt Sublime Text đã kết thúc thành công.
Yêu cầu 2: Sử dụng phần mềm Sublime Text để soạn một văn bản HTML sao cho khi mở văn bản bằng trình duyệt web, trên màn hình hiển thị dòng chữ: "Chủ đề F: Tạo trang web"
Hướng dẫn thực hiện:
Bước .1 Khởi động Sublime Text bằng cách nháy đúp chuột vào biểu tượng phần mềm.
Bước .2 Trong màn hình làm việc, soạn thảo nội dung văn bản HTML.
Bước .3 Chọn File\Save, ghi lưu tệp với tên "trangwebdautien.html".
Bước .4 Mở tệp bằng trình duyệt web, xem kết quả.

Tóm tắt bài học
• Văn bản HTML định nghĩa các phần tử để xác định nội dung và cấu trúc của trang web. Phần tử thường được khai báo bắt đầu bằng thẻ mở và kết thúc bằng thẻ đóng.
• Các phần tử html, head, body là các thành phần cơ bản của văn bản HTML.
• Văn bản HTML dễ dàng được tạo bằng các phần mềm hỗ trợ soạn thảo văn bản.

"""

Fixing prompt to improve result

In [60]:
def remove_multi_spaces(text):
    return re.sub(r' +', ' ', text)

def remove_brackets(text):
    return re.sub(r'\((.*?)\)', lambda L: '', text)

def remove_square_brackets(text):
    return re.sub(r'\[(.*?)\]', lambda L: '', text)

def remove_multi_downline(text):
    return re.sub('\n+','\n',text)

In [61]:
input_texts = remove_multi_downline(input_texts)
input_texts = remove_multi_spaces(input_texts)

In [56]:
# def create_sequences(data, tokenizer, max_length=128, stride=25):
#     input_sequences = []
#     for line in data.split('\n'):
#         tokens = tokenizer.tokenize(line)
#         for i in range(0, len(tokens), stride):
#             n_gram_tokens = tokens[i:i+max_length]
#             if len(n_gram_tokens) > 1:
#                 n_gram_text = tokenizer.convert_tokens_to_string(n_gram_tokens)
#                 input_sequences.append(n_gram_text)
#     return input_sequences

def split_into_sentences(text):
    sentence_endings = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s')
    sentences = sentence_endings.split(text)
    return sentences

def create_sequences_from_sentences(data, tokenizer, max_length=128):
    input_sequences = []
    sentences = split_into_sentences(data)
    current_sequence = []

    for sentence in sentences:
        current_sequence.append(sentence)
        tokenized_sequence = tokenizer.tokenize(' '.join(current_sequence))
        if len(tokenized_sequence) > max_length:
            current_sequence.pop()
            input_sequences.append(' '.join(current_sequence))
            current_sequence = [sentence]

    if current_sequence:
        input_sequences.append(' '.join(current_sequence))

    return input_sequences

In [27]:
#improve later
def split_context_according_to_desired_count(context: str, desired_count: int) -> List[str]:
    sents = sent_tokenize(context)
    sent_ratio = len(sents) / desired_count

    context_splits = []

    if sent_ratio < 1:
        return sents
    else:
        take_sents_count = int(sent_ratio + 1)

        start_sent_index = 0

        while start_sent_index < len(sents):
            context_split = ' '.join(sents[start_sent_index: start_sent_index + take_sents_count])
            context_splits.append(context_split)
            start_sent_index += take_sents_count - 1

    return context_splits

In [52]:
model = AutoModelForSeq2SeqLM.from_pretrained("my_vit5_finetuned")
tokenizer = AutoTokenizer.from_pretrained("my_vit5_finetuned")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

def generate_text(context, max_length=128):
    inputs = tokenizer.encode_plus(context, return_tensors="pt", max_length=128, truncation=True).to(device)

    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=128,
        top_p=1,
        top_k=50,
        temperature=1,
        repetition_penalty=1.2,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(context)
    print(generated_text)


In [63]:
# context_split = split_context_according_to_desired_count(input_texts, 10)

context_split = create_sequences_from_sentences(input_texts, tokenizer)

for i in range(5):
    generate_text(context_split[i])
    print()



1. Ngôn ngữ đánh dấu siêu văn bản
Thông thường, một website gồm một số trang web tĩnh và một số trang web động. Trang web tĩnh có nội dung không thay đổi mỗi khi người dùng truy cập. Ngược lại, nội dung trang web động có thể thay đổi tuỳ theo yêu cầu của người dùng. Tìm hiểu xong chủ đề này, em sẽ tạo được các trang web tĩnh. Có nhiều cách để tạo trang web. Bên cạnh cách sử dụng phần mềm có sẵn như: Dreamweaver, Mobirise,.., em có thể tạo trang web bằng ngôn ngữ chuyên dụng.
Ngôn ngữ nào có thể thay đổi mỗi khi người dùng truy cập? sep> ngôn ngữ đánh dấu siêu văn bản

Ngôn ngữ đánh dấu siêu văn bản (HyperText Markup Language - HTML) là ngôn ngữ chuyên dụng dùng để tạo trang web. Thông qua các phần tử của mình, HTML cho phép khai báo các thành phần của trang web như tiêu đề mục, đoạn văn, bảng biểu, hình ảnh, âm thanh và các siêu liên kết,... Phần đầu của văn bản được xác định thông qua phần tử head.
Ngôn ngữ đánh dấu siêu văn bản là gì? sep> HTML

Nội dung phần tử head được viết trong

In [70]:
input_texts = \
r"""

Bài 3: THỰC HÀNH ĐỊNH DẠNG VĂN BẢN VÀ TẠO SIÊU LIÊN KẾT
Học xong bài này, em sẽ:
Tạo được trang web đơn giản với các đoạn văn bản và các tiêu đề mục.
Làm nổi bật được nội dung văn bản trên màn hình trình duyệt web.
Tạo được siêu liên kết.

Nhiệm vụ 1. Tạo tiêu đề mục cho trang web giới thiệu về bản thân
Yêu cầu: Soạn thảo nội dung trang web giới thiệu bản thân và lưu tệp văn bản
HTML với tên "Bai3-NV1.html". Màn hình trình duyệt web cần hiển thị:
- Tiêu đề trang web là: Trang web cá nhân.
- Dòng đầu tiên viết nội dung: Trang web cá nhân của <Tên của em> được trình
bày với tiêu đề mục Heading .1 Ví dụ: Trang web cá nhân của Thanh Uyên.
- Các dòng tiếp theo là tiêu đề mục Heading 2 với các mục Thông tin cá nhân,
Sở thích, mỗi mục viết trên một dòng.

Hướng dẫn thực hiện:
Bước 1. Tạo tệp "Bai3-NV1.html".
- Mở phần mềm Sublime Text.
- Mở tệp mới bằng cách chọn File\New File.
- Lưu tệp với tên là "Bai3-NV1.html", thực hiện soạn thảo.
Bước 2. Tạo cấu trúc và khai báo phần tử head cho tệp "Bai3-NV1.html".
- Dòng đầu tiên soạn <!DOCTYPE html> để khai báo sử dụng phiên bản HTML5.
- Khai báo phần tử html bằng cặp thẻ: <html</html.
- Trong nội dung phần tử html:
+ Khai báo phần tử head bằng cặp thẻ: <head</head.
+ Khai báo phần tử body bằng cặp thẻ: <body></body>.
- Trong nội dung phần tử head:
+ Khai báo tiêu đề trang web bằng phân tử title.
+ Thiết lập thuộc tính charset của phần tử meta để hiến thị đúng tiếng Việt trên trình duyệt bằng khai báo <meta charset= "utf-8">.
Bước 3. Soạn nội dung phần tử body cho tệp "Bai3-NV1.html".
- Sử dụng phần tử h1 để trình bày tiêu đề mục Heading 1.
- Sử dụng phần tử h2 để trình bày tiêu đề mục Heading 2.
Bước 4. Ghi lưu, mở tệp bằng trình duyệt web và xem kết quả.
Lưu ý: Để thêm chú thích vào văn bản HTML, em viết chú thích trong cặp thẻ <!-- và -->. Các chú thích sẽ không hiến thị trên trình duyệt web.
Nhiệm vụ 2. Làm nổi bật nội dung cho trang web giới thiệu về bản thân
Yêu cầu:
1) Soạn nội dung chi tiết cho mỗi mục đã có tiêu đề trong trang web giới thiệu bản thân ở tệp "Bai3-NV1.html"
- Mục Thông tin cá nhân cần có các thông tin: Họ và tên, Sinh năm, Quê quán.
- Mục Sở thích, nêu các sở thích của mình (ví dụ, Đọc sách: Các tác phẩm của nhà văn Nguyễn Nhật Ánh; Thể thao: Chơi bóng chuyền, Chơi cầu lông).
2) Trình bày trang web như sau:
- Các nội dung trong từng mục được trình bày dưới dạng các đoạn văn bản.
- Nội dung trong mục Sở thích được in nghiêng.

Hướng dẫn thực hiện:
Bước 1. Mở tệp "Bai3-NV1.html", ghi lưu tệp với tên mới là "Bai3-NV2 html"
Bước 2. Cập nhật nội dung phân tử body cho tệp "Bai3-NV2.html"
- Chèn dòng mới vào sau dòng khai báo tiêu đề mục "Thông tin cá nhân", sử dụng phần tử p để tạo đoạn văn ghi nội dung "Họ và tên: <Tên của em >", "Sinh năm: <Năm sinh của em>", "Quê quán: <Quê quán của em ">
- Chèn dòng mới vào sau dòng khai báo tiêu dề mục "Sở thích" tương tự như trên.
Bước 3. Làm nổi bật nội dung trang web.
Trong nội dung phần tử body: Kết hợp sử dụng phần tử p và phần tử em để in nghiêng các nội dung cụ thể.
Bước 4. Ghi lưu, mở tệp bằng trình duyệt web và xem kết quả.

Nhiệm vụ 3. Tạo siêu liên kết
Yêu cầu: Trong trang web giới thiệu cá nhân vừa tạo được ở Nhiệm vụ 2, em hãy tạo siêu liên kết để cung cấp thêm thông tin về Quê quán. Kết quả cần có là khi nháy chuột vào tên tỉnh/thành phố trong mục Quê quán, trang web cổng thông tin điện tử của quê hương em sẽ được mở ra.

Hướng dẫn thực hiện:
Bước 1. Mở tệp "Bai3-NV2.html", ghi lưu với tên mới là "Bai3-NV3.html".
Bước 2. Sửa nội dung dòng thông tin Quê quán để tạo siêu liên kết bằng cách sử dụng phần tử a.
Ví dụ, khai báo sau sẽ tạo siêu liên kết đến trang web cổng thông tin điện tử thành phố Hà Nội
<p> Quê quán: <a href="https://www.hanoi.gov.vn/"> Hà Nội</a></p>
Lưu ý: Các tỉnh, thành phố đều có trang web cổng thông tin điện t ử với tên
dạng <tentinh>.gov.vn. (Ví dụ: thanhphohaiphong.gov.vn, hochiminhcity.gov.vn,...).
Bước 3. Ghi lưu, mở tệp bằng trình duyệt web và xem kết quả.

"""

In [71]:
context_split = create_sequences_from_sentences(input_texts, tokenizer)

for i in range(5):
    generate_text(context_split[i])
    print()



Bài 3: THỰC HÀNH ĐỊNH DẠNG VĂN BẢN VÀ TẠO SIÊU LIÊN KẾT
Học xong bài này, em sẽ:
Tạo được trang web đơn giản với các đoạn văn bản và các tiêu đề mục. Làm nổi bật được nội dung văn bản trên màn hình trình duyệt web. Tạo được siêu liên kết. 
Nhiệm vụ 1. Tạo tiêu đề mục cho trang web giới thiệu về bản thân
Yêu cầu: Soạn thảo nội dung trang web giới thiệu bản thân và lưu tệp văn bản
HTML với tên "Bai3-NV1.html".
Điều gì có thể tạo ra siêu liên kết? sep> Tạo được trang web đơn giản với các đoạn văn bản và các tiêu đề mục

Màn hình trình duyệt web cần hiển thị:
- Tiêu đề trang web là: Trang web cá nhân. - Dòng đầu tiên viết nội dung: Trang web cá nhân của <Tên của em> được trình
bày với tiêu đề mục Heading .1 Ví dụ: Trang web cá nhân của Thanh Uyên. - Các dòng tiếp theo là tiêu đề mục Heading 2 với các mục Thông tin cá nhân,
Sở thích, mỗi mục viết trên một dòng. 
Hướng dẫn thực hiện:
Bước 1. Tạo tệp "Bai3-NV1.html". - Mở phần mềm Sublime Text.
Tên của trang web cá nhân của Thanh Uyên là gì